In [2]:
from biocrnpyler import *
from genelet import *
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem, createBasicSubsystem, SimpleModel, SimpleReaction
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import libsbml
import bokeh.io
import bokeh.plotting

In [3]:
ss1 = createSubsystem('NOR_CRN.xml')
ss2 = createSubsystem('rI4_external_reservoir.xml')

# Create a simple IPTG membrane where IPTG goes in an out of the membrane via a reversible reaction
mb = createSubsystem('membrane.xml', membrane = True)

# cell_2 =  System('cell_2',ListOfInternalSubsystems = [ss1],
#                  ListOfExternalSubsystems = [ss2],
#                  ListOfMembraneSubsystems = [mb2])
cell_1 = System('cell_1')
cell_1.setInternal(ss1)
cell_1.setExternal(ss2)
cell_1.setMembrane(mb)
cell_1_model = cell_1.getModel()

#cell_1_model.setSpeciesAmount('rna_I4', 0, compartment = 'cell_1_external')

cell_1_model.setSpeciesAmount('Sw1_OFF', 2000, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_A1', 0, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('rna_I1', 0, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('Sw2_OFF', 2000, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_A2', 0, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('rna_I2', 0, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('Sw3_OFF', 2000, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_A3', 2000, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('rna_I3', 0, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('protein_RNAseH', 10, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('protein_RNAP', 500, compartment = 'cell_1_internal')


cell_1_model.writeSBML('cell_1_model.xml')

The subsystem from membrane.xml has multiple compartments


1

In [4]:
timepoints = np.linspace(0, 1000, 1000)
R, _ = cell_1_model.simulateWithBioscrape(timepoints)
timepoints = timepoints/60



Sw1_ON_id = cell_1_model.getSpeciesByName('complex_Sw1_ON').getId()
Sw2_ON_id = cell_1_model.getSpeciesByName('complex_Sw2_ON').getId()
Sw3_ON_id = cell_1_model.getSpeciesByName('complex_Sw3_ON').getId()
Sw1_OFF_id = cell_1_model.getSpeciesByName('Sw1_OFF').getId()
Sw2_OFF_id = cell_1_model.getSpeciesByName('Sw2_OFF').getId()
Sw3_OFF_id = cell_1_model.getSpeciesByName('Sw3_OFF').getId()
rna_I3_id = cell_1_model.getSpeciesByName('rna_I3').getId()
dna_A3_id = cell_1_model.getSpeciesByName('dna_A3').getId()
AI_id = cell_1_model.getSpeciesByName('complex_Sw3_AI').getId()


rna_I4_i_id = cell_1_model.getSpeciesByName('rna_I4', compartment = 'cell_1_internal').getId()
rna_I4_e_id = cell_1_model.getSpeciesByName('rna_I4', compartment = 'cell_1_external').getId()

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
p.circle(timepoints, R[Sw1_OFF_id], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R[Sw1_ON_id], legend_label = "ON switch 1", color = "red")

s = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
s.circle(timepoints, R[Sw2_OFF_id], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R[Sw2_ON_id], legend_label = "ON switch 2" , color = "red")

q = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
q.circle(timepoints, R[Sw3_OFF_id], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R[Sw3_ON_id], legend_label = "ON switch 3" , color = "red")

# NOR gate activators and inhibitors of importance

r = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
r.circle(timepoints, R[rna_I3_id], legend_label = "Inhibitor 3", color = "red")
r.circle(timepoints, R[rna_I4_i_id], legend_label = "Inhibitor 4 internal", color = "blue")
r.circle(timepoints, R[rna_I4_e_id], legend_label = "Inhibitor 4 external", color = "purple")
#r.circle(timepoints, R[dna_A3_id], legend_label = "Activator 3", color = "orange")
#r.circle(timepoints, R[AI_id], legend_label = "AI 3", color = "black")


r.legend.click_policy="hide"

#s = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
#s.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "red")
#s.legend.click_policy="hide"

bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")



/home/nazmus/anaconda3/lib/python3.7/site-packages/bioscrape/sbmlutil.py:90: UserWarning: SBML model contains reversible reaction!
Please check rate expressions and ensure they are non-negative before doing stochastic simulations.
  'Please check rate expressions and ensure they are non-negative before doing '+
/home/nazmus/anaconda3/lib/python3.7/site-packages/bioscrape/sbmlutil.py:209: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +


Loading BokehJS ...

In [5]:
mb2 = createSubsystem('membrane_detailed.xml', membrane = True)

cell_2 = System('cell_2')
cell_2 =  System('cell_2',ListOfInternalSubsystems = [ss1],
                 ListOfExternalSubsystems = [ss2],
                 ListOfMembraneSubsystems = [mb2])
cell_2_model = cell_2.getModel()

#cell_1_model.setSpeciesAmount('rna_I4', 0, compartment = 'cell_1_external')
cell_2_model.setSpeciesAmount('transport_protein', 100, compartment = 'cell_2_internal')

cell_2_model.setSpeciesAmount('Sw1_OFF', 2000, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('dna_A1', 0, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('rna_I1', 0, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('Sw2_OFF', 2000, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('dna_A2', 0, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('rna_I2', 0, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('Sw3_OFF', 2000, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('dna_A3', 2000, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('rna_I3', 0, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('protein_RNAseH', 10, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('protein_RNAP', 500, compartment = 'cell_2_internal')


cell_2_model.writeSBML('cell_2_model.xml')

The subsystem from membrane_detailed.xml has multiple compartments


1

In [6]:
timepoints = np.linspace(0, 1000, 1000)
R, _ = cell_2_model.simulateWithBioscrape(timepoints)
timepoints = timepoints/60



Sw1_ON_id = cell_2_model.getSpeciesByName('complex_Sw1_ON').getId()
Sw2_ON_id = cell_2_model.getSpeciesByName('complex_Sw2_ON').getId()
Sw3_ON_id = cell_2_model.getSpeciesByName('complex_Sw3_ON').getId()
Sw1_OFF_id = cell_2_model.getSpeciesByName('Sw1_OFF').getId()
Sw2_OFF_id = cell_2_model.getSpeciesByName('Sw2_OFF').getId()
Sw3_OFF_id = cell_2_model.getSpeciesByName('Sw3_OFF').getId()
rna_I3_id = cell_2_model.getSpeciesByName('rna_I3').getId()
dna_A3_id = cell_2_model.getSpeciesByName('dna_A3').getId()
AI_id = cell_2_model.getSpeciesByName('complex_Sw3_AI').getId()


rna_I4_i_id = cell_2_model.getSpeciesByName('rna_I4', compartment = 'cell_2_internal').getId()
rna_I4_e_id = cell_2_model.getSpeciesByName('rna_I4', compartment = 'cell_2_external').getId()

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
p.circle(timepoints, R[Sw1_OFF_id], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R[Sw1_ON_id], legend_label = "ON switch 1", color = "red")

s = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
s.circle(timepoints, R[Sw2_OFF_id], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R[Sw2_ON_id], legend_label = "ON switch 2" , color = "red")

q = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
q.circle(timepoints, R[Sw3_OFF_id], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R[Sw3_ON_id], legend_label = "ON switch 3" , color = "red")

# NOR gate activators and inhibitors of importance

r = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)',
    y_axis_label='Concentration (uM)')
r.circle(timepoints, R[rna_I3_id], legend_label = "Inhibitor 3", color = "red")
r.circle(timepoints, R[rna_I4_i_id], legend_label = "Inhibitor 4 internal", color = "blue")
r.circle(timepoints, R[rna_I4_e_id], legend_label = "Inhibitor 4 external", color = "purple")
#r.circle(timepoints, R[dna_A3_id], legend_label = "Activator 3", color = "orange")
#r.circle(timepoints, R[AI_id], legend_label = "AI 3", color = "black")


r.legend.click_policy="hide"

#s = bokeh.plotting.figure(plot_width=300, plot_height=300, x_axis_label='Time (min)')
#s.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "red")
#s.legend.click_policy="hide"

bokeh.io.show(row(p, s, q, r))
warnings.filterwarnings("ignore")



Loading BokehJS ...